In [1]:
import pandas as pd
import numpy as np
import yfinance as yf

In [2]:
firm_data = pd.read_csv('../inputs/Firm_data.csv')
data_desc = pd.read_csv('../data_description/Firm_data_descriptions.csv')

In [3]:
data_desc['Variable Name'] = data_desc['Variable Name'].str.lower()
data_desc = data_desc[['Variable Name','Description']]
column_names = pd.DataFrame(firm_data.columns)
column_names = column_names.rename(columns = {column_names.columns[0]: 'Variable Name'})
data_definitions = column_names.merge(data_desc, 
                how='left',
                on='Variable Name',
                indicator=False,
                validate='1:1')

In [4]:
data_definitions

,Variable Name,Description
0,gvkey,GVKEY -- Global Company Key (GVKEY)
1,datadate,NaN
2,fyear,FYEAR -- Data Year - Fiscal (FYEAR)
3,indfmt,NaN
4,consol,NaN
5,popsrc,NaN
6,datafmt,NaN
7,tic,Ticker Symbol (TIC)
8,cusip,CUSIP (CUSIP)
9,conm,Company Name (CONM)


### Investment Opportunities

**Market-to-book**

In [5]:
def mtb(tic, year):
    market_value = firm_data.query('tic == @tic & fyear == @year')['mkvalt'].iloc[0]
    book_value = firm_data.query('tic == @tic & fyear == @year')['ceql'].iloc[0]
    value = market_value/book_value
    return value

**Tobin's Q**

For Tobin's Q -- is "debt" just liabilities? or just debt? (I included both in the dataset)

In [6]:
def tobinsQ(tic, year):
    price = firm_data.query('tic == @tic & fyear == @year')['prcc_f'].iloc[0]
    so = firm_data.query('tic == @tic & fyear == @year')['csho'].iloc[0]
    assets = firm_data.query('tic == @tic & fyear == @year')['at'].iloc[0]
    equity = firm_data.query('tic == @tic & fyear == @year')['seq'].iloc[0]
    return ((price * so) + assets - equity) / assets

**R&D Scaled**

In [7]:
# def rdScaled(tic, year):
#     rd = firm_data.query('tic == @tic & fyear == @year')['xrd']
#     assets = firm_data.query('tic == @tic & fyear == @year')['at']
#     return rd/assets

**Advertising Scaled**

In [8]:
# def advScaled(tic, year):
#     adv = firm_data.query('tic == @tic & fyear == @year')['xad']
#     assets = firm_data.query('tic == @tic & fyear == @year')['at']
#     return adv/assets

**Liquidity**

In [9]:
def liquidity(tic, year):
    currA = firm_data.query('tic == @tic & fyear == @year')['act'].iloc[0]
    currL = firm_data.query('tic == @tic & fyear == @year')['lct'].iloc[0]
    assets = firm_data.query('tic == @tic & fyear == @year')['at'].iloc[0]
    return (currA-currL)/assets

In [10]:
liquidity('AAPL',2012)

0.10854575608869502

### Firm Complexity

**Sales per Firm per Year**

In [11]:
sales = firm_data[['tic','fyear','ni']].copy()
sales # In millions per year

,tic,fyear,ni
0,AAL,2010,-471.0
1,AAL,2011,-1979.0
2,AAL,2012,-1876.0
3,AAL,2013,-1834.0
4,AAL,2014,2882.0
...,...,...,...
4877,ALLE,2015,153.9
4878,ALLE,2016,229.1
4879,ALLE,2017,273.3
4880,ALLE,2018,434.9


In [12]:
# To query:
def netIncome(tic, year):
    return firm_data.query('tic == @tic & fyear == @year')['ni'].iloc[0]

In [13]:
netIncome('AAL', 2012)

-1876.0

**Number of Employees per Firm per Year**

In [14]:
employees = firm_data[['tic','fyear','emp']].copy()
employees # In thousands per year

,tic,fyear,emp
0,AAL,2010,78.25
1,AAL,2011,80.10
2,AAL,2012,77.75
3,AAL,2013,110.40
4,AAL,2014,113.30
...,...,...,...
4877,ALLE,2015,9.40
4878,ALLE,2016,9.40
4879,ALLE,2017,10.00
4880,ALLE,2018,11.00


In [15]:
# To query:
def numEmployees(tic, year):
    return firm_data.query('tic == @tic & fyear == @year')['emp'].iloc[0]

In [16]:
numEmployees('AAL', 2012)

77.75

**Market Value per Firm per Year**

In [17]:
mv = firm_data[['tic','fyear','mkvalt']].copy()
mv # In millions per year

,tic,fyear,mkvalt
0,AAL,2010,2597.5755
1,AAL,2011,117.3438
2,AAL,2012,266.5571
3,AAL,2013,6591.9923
4,AAL,2014,37405.5843
...,...,...,...
4877,ALLE,2015,6327.7267
4878,ALLE,2016,6097.5360
4879,ALLE,2017,7563.1327
4880,ALLE,2018,7543.5153


In [18]:
# To query:
def marketValue(tic, year):
    return firm_data.query('tic == @tic & fyear == @year')['mkvalt'].iloc[0]

In [19]:
marketValue('AAL', 2012)

266.5571

### Monitoring

**Leverage (Debt / Equity)**

In [20]:
def dte(tic, year):
    debt = firm_data.query('tic == @tic & fyear == @year')['lt'].iloc[0]
    equity = firm_data.query('tic == @tic & fyear == @year')['seq'].iloc[0]
    return debt/equity

**Assets in Place**

In [21]:
def assetsInPlace(tic, year):
    ppe = firm_data.query('tic == @tic & fyear == @year')['ppegt'].iloc[0]
    assets = firm_data.query('tic == @tic & fyear == @year')['at'].iloc[0]
    return ppe/assets

**CapEx Scaled by Assets**

In [22]:
def capexAssets(tic, year):
    capx = firm_data.query('tic == @tic & fyear == @year')['capx'].iloc[0]
    assets = firm_data.query('tic == @tic & fyear == @year')['at'].iloc[0]
    return capx/assets

In [23]:
# All variables but Tobin's Q

firm_date_data = firm_data[['tic','fyear']].copy()
firm_date_data = firm_date_data.dropna()

**Return on Assets**

In [24]:
def roa(tic, year):
    ni = firm_data.query('tic == @tic & fyear == @year')['ni'].iloc[0]
    assets = firm_data.query('tic == @tic & fyear == @year')['at'].iloc[0]
    return ni/assets

In [25]:
firm_date_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4882 entries, 0 to 4881
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tic     4882 non-null   object
 1   fyear   4882 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 76.4+ KB


In [26]:
null_mask = firm_date_data.isnull().any(axis=1)
print(firm_date_data[null_mask])
firm_date_data

Empty DataFrame
Columns: [tic, fyear]
Index: []


,tic,fyear
0,AAL,2010
1,AAL,2011
2,AAL,2012
3,AAL,2013
4,AAL,2014
...,...,...
4877,ALLE,2015
4878,ALLE,2016
4879,ALLE,2017
4880,ALLE,2018


In [27]:
# commented for runs

# firm_date_data['liquidity'] = firm_date_data.apply(lambda x: liquidity(x['tic'], x['fyear']), axis=1)
# firm_date_data['net_income'] = firm_date_data.apply(lambda x: netIncome(x['tic'], x['fyear']), axis=1)
# firm_date_data['num_employees'] = firm_date_data.apply(lambda x: numEmployees(x['tic'], x['fyear']), axis=1)
# firm_date_data['market_value'] = firm_date_data.apply(lambda x: marketValue(x['tic'], x['fyear']), axis=1)
# firm_date_data['debt_to_equity'] = firm_date_data.apply(lambda x: dte(x['tic'], x['fyear']), axis=1)
# firm_date_data['assets_in_place'] = firm_date_data.apply(lambda x: assetsInPlace(x['tic'], x['fyear']), axis=1)
# firm_date_data['capex_by_assets'] = firm_date_data.apply(lambda x: capexAssets(x['tic'], x['fyear']), axis=1)
# firm_date_data['return_on_equity'] = firm_date_data.apply(lambda x: roa(x['tic'], x['fyear']), axis=1)

In [28]:
# commented for runs

# corr_measure = firm_date_data[['tic','fyear']]
# corr_measure['market_to_book'] = corr_measure.apply(lambda x: mtb(x['tic'], x['fyear']), axis=1)
# corr_measure['TobinsQ'] = corr_measure.apply(lambda x: tobinsQ(x['tic'], x['fyear']), axis=1)

In [29]:
# Data is reported on 12/31 for the year in question, so +1 sets it to year start
firm_date_data['fyear'] = firm_date_data['fyear']+1

In [30]:
firm_date_data.to_csv('../input_data_clean/director_comp_deter.csv', index=False)

In [31]:
firm_date_data

,tic,fyear
0,AAL,2011
1,AAL,2012
2,AAL,2013
3,AAL,2014
4,AAL,2015
...,...,...
4877,ALLE,2016
4878,ALLE,2017
4879,ALLE,2018
4880,ALLE,2019


### Firm Performance

In [32]:
firm_data

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,cusip,conm,...,teq,xad,xrd,cik,costat,dvpsp_f,mkvalt,prcc_f,gsector,naics
0,1045,2010-12-31,2010,INDL,C,D,STD,AAL,02376R102,AMERICAN AIRLINES GROUP INC,...,-3945.0,165.0,NaN,6201,A,0.00,2597.5755,7.790,20,481111
1,1045,2011-12-31,2011,INDL,C,D,STD,AAL,02376R102,AMERICAN AIRLINES GROUP INC,...,-7111.0,186.0,NaN,6201,A,0.00,117.3438,0.350,20,481111
2,1045,2012-12-31,2012,INDL,C,D,STD,AAL,02376R102,AMERICAN AIRLINES GROUP INC,...,-7987.0,153.0,NaN,6201,A,0.00,266.5571,0.795,20,481111
3,1045,2013-12-31,2013,INDL,C,D,STD,AAL,02376R102,AMERICAN AIRLINES GROUP INC,...,-2731.0,166.0,NaN,6201,A,0.00,6591.9923,25.250,20,481111
4,1045,2014-12-31,2014,INDL,C,D,STD,AAL,02376R102,AMERICAN AIRLINES GROUP INC,...,2021.0,100.0,NaN,6201,A,0.20,37405.5843,53.630,20,481111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4877,316056,2015-12-31,2015,INDL,C,D,STD,ALLE,G0176J109,ALLEGION PLC,...,29.7,NaN,45.2,1579241,A,0.40,6327.7267,65.920,20,332510
4878,316056,2016-12-31,2016,INDL,C,D,STD,ALLE,G0176J109,ALLEGION PLC,...,116.4,NaN,47.3,1579241,A,0.48,6097.5360,64.000,20,332510
4879,316056,2017-12-31,2017,INDL,C,D,STD,ALLE,G0176J109,ALLEGION PLC,...,405.5,NaN,48.3,1579241,A,0.64,7563.1327,79.560,20,332510
4880,316056,2018-12-31,2018,INDL,C,D,STD,ALLE,G0176J109,ALLEGION PLC,...,654.0,NaN,54.4,1579241,A,0.84,7543.5153,79.710,20,332510


### Profitability Performance

In [33]:
def EBITDAMargin(tic, year):
    ebitda = firm_data.query('tic == @tic & fyear == @year')['ebitda'].iloc[0]
    revenue = firm_data.query('tic == @tic & fyear == @year')['revt'].iloc[0]
    return ebitda/revenue

In [34]:
def niRev(tic, year):
    netIncome = firm_data.query('tic == @tic & fyear == @year')['ni'].iloc[0]
    revenue = firm_data.query('tic == @tic & fyear == @year')['revt'].iloc[0]
    return netIncome/revenue

In [35]:
def roe(tic, year):
    netIncome = firm_data.query('tic == @tic & fyear == @year')['ni'].iloc[0]
    equity = firm_data.query('tic == @tic & fyear == @year')['seq'].iloc[0]
    return netIncome/equity

### Market Value Performance

In [36]:
def eps(tic, year):
    netIncome = firm_data.query('tic == @tic & fyear == @year')['ni'].iloc[0]
    prefDiv = firm_data.query('tic == @tic & fyear == @year')['dvp'].iloc[0]
    so = firm_data.query('tic == @tic & fyear == @year')['csho'].iloc[0]
    return (netIncome-prefDiv)/so

In [37]:
# takes a long time

def stockPrice(tic, year):
    start_date = str(int(year)) + "-01-01"
    end_date = str(int(year)) + "-12-31"
    try:
        ticker_data = yf.download(tic, start=start_date, end=end_date, progress=False)
        start_price = ticker_data['Adj Close'][0]
        end_price = ticker_data['Adj Close'][-1]
        change = (end_price/start_price)-1
        return change
    except IndexError:
        return np.nan

In [38]:
def divYield(tic, year):
    dividend = firm_data.query('tic == @tic & fyear == @year')['dvpsp_f'].iloc[0]
    price = firm_data.query('tic == @tic & fyear == @year')['prcc_f'].iloc[0]
    return dividend/price

In [39]:
# takes a long time

def volatility(tic, year):
    start_date = str(int(year)) + "-01-01"
    end_date = str(int(year)) + "-12-31"
    try:
        ticker_data = yf.download(tic, start=start_date, end=end_date, progress=False)
        daily_ret = ticker_data['Adj Close'].pct_change()[1:]
        vol = np.sqrt(252)*daily_ret.std()
        return vol
    except IndexError:
        return np.nan

In [40]:
def mva(tic, year):
    marketVal = firm_data.query('tic == @tic & fyear == @year')['mkvalt'].iloc[0]
    equity = firm_data.query('tic == @tic & fyear == @year')['seq'].iloc[0]
    return marketVal-equity

### Growth Performance

In [41]:
firm_growth_data = pd.read_csv('../Input_data/growth_perf.csv')

In [42]:
def assetg(tic, year):
    prior_year = year-1
    try: # try in case data not available for prior year or current year
        yrminus1 = firm_growth_data.query('tic == @tic & fyear == @prior_year')['at'].iloc[0]
        yr = firm_growth_data.query('tic == @tic & fyear == @year')['at'].iloc[0]
        return (yr/yrminus1)-1
    except IndexError: # if data not available, return NaN
        return np.nan

In [43]:
def revenueg(tic, year):
    prior_year = year-1
    try:
        yrminus1 = firm_growth_data.query('tic == @tic & fyear == @prior_year')['revt'].iloc[0]
        yr = firm_growth_data.query('tic == @tic & fyear == @year')['revt'].iloc[0]
        return (yr/yrminus1)-1
    except IndexError:
        return np.nan

In [44]:
def incomeg(tic, year):
    prior_year = year-1
    try:
        yrminus1 = firm_growth_data.query('tic == @tic & fyear == @prior_year')['ni'].iloc[0]
        yr = firm_growth_data.query('tic == @tic & fyear == @year')['ni'].iloc[0]
        return (yr/yrminus1)-1
    except IndexError:
        return np.nan

In [45]:
def employeeg(tic, year):
    prior_year = year-1
    try:
        yrminus1 = firm_growth_data.query('tic == @tic & fyear == @prior_year')['emp'].iloc[0]
        yr = firm_growth_data.query('tic == @tic & fyear == @year')['emp'].iloc[0]
        return (yr/yrminus1)-1
    except IndexError:
        return np.nan

### Building the dataframe

In [46]:
firm_performance = firm_data[['tic','fyear']].copy()
firm_performance = firm_performance.dropna()

In [47]:
# Profitability
# ROA = NI / assets
firm_performance['roa'] = firm_performance.apply(lambda x: roa(x['tic'], x['fyear']), axis=1)
# EBITDA Margin = EBITDA/total revenue
firm_performance['EBITDA_margin'] = firm_performance.apply(lambda x: EBITDAMargin(x['tic'], x['fyear']), axis=1)
# Net income / Revenue
firm_performance['NI_rev'] = firm_performance.apply(lambda x: niRev(x['tic'], x['fyear']), axis=1)
# ROE = Net income / Shareholders equity
firm_performance['roe'] = firm_performance.apply(lambda x: roe(x['tic'], x['fyear']), axis=1)

# Market value
# EPS = (net income - preferred dividends)/ shares outstanding
firm_performance['eps'] = firm_performance.apply(lambda x: eps(x['tic'], x['fyear']), axis=1)
# Changes in stock price - pull from Yahoo Finance
firm_performance['chng_price'] = firm_performance.apply(lambda x: stockPrice(x['tic'], x['fyear']), axis=1)
# Dividend yield = dividends per share / price per share
firm_performance['div_yield'] = firm_performance.apply(lambda x: divYield(x['tic'], x['fyear']), axis=1)
# Volatility = standard deviation multiplied by the square root of the number of periods of time (252 trading days in US)
firm_performance['vol'] = firm_performance.apply(lambda x: volatility(x['tic'], x['fyear']), axis=1)
# MVA = Market Value of Shares - Shareholders equity (https://corporatefinanceinstitute.com/resources/valuation/market-value-added-mva/)
firm_performance['mva'] = firm_performance.apply(lambda x: mva(x['tic'], x['fyear']), axis=1)
# Tobin's Q = (Market Value of common stock + book value of total assets - book value of common equity)/book value total assets (https://funginstitute.berkeley.edu/wp-content/uploads/2013/12/Younge-Marx-2012-Working-Paper-The-Market-Value-of-Knowledge-Protection-Jan-10-2012.pdf)
    # ((PRCC_F * CSHO) + AT – CEQ ) / AT
firm_performance['tobinsQ'] = firm_performance.apply(lambda x: tobinsQ(x['tic'], x['fyear']), axis=1)

# Growth Performance
# Asset growth
firm_performance['asset_g'] = firm_performance.apply(lambda x: assetg(x['tic'], x['fyear']), axis=1)
# Total revenue growth
firm_performance['revenue_g'] = firm_performance.apply(lambda x: revenueg(x['tic'], x['fyear']), axis=1)
# Net income growth
firm_performance['ni_g'] = firm_performance.apply(lambda x: incomeg(x['tic'], x['fyear']), axis=1)
# Employee growth
firm_performance['employee_g'] = firm_performance.apply(lambda x: employeeg(x['tic'], x['fyear']), axis=1)


1 Failed download:
- BRK.B: No timezone found, symbol may be delisted

1 Failed download:
- BRK.B: No timezone found, symbol may be delisted

1 Failed download:
- BRK.B: No timezone found, symbol may be delisted

1 Failed download:
- BRK.B: No timezone found, symbol may be delisted

1 Failed download:
- BRK.B: No timezone found, symbol may be delisted

1 Failed download:
- BRK.B: No timezone found, symbol may be delisted

1 Failed download:
- BRK.B: No timezone found, symbol may be delisted

1 Failed download:
- BRK.B: No timezone found, symbol may be delisted

1 Failed download:
- BRK.B: No timezone found, symbol may be delisted

1 Failed download:
- BRK.B: No timezone found, symbol may be delisted

1 Failed download:
- BF.B: No data found for this date range, symbol may be delisted

1 Failed download:
- BF.B: No data found for this date range, symbol may be delisted

1 Failed download:
- BF.B: No data found for this date range, symbol may be delisted

1 Failed download:
- BF.B: No d

KeyError: 'APH'

In [ ]:
firm_performance.info()

In [ ]:
firm_performance

In [ ]:
firm_performance.to_csv('../outputs/Firm_Performance.csv', index=False)